Importing open-cv, numpy and harcascade file for face detction

In [ ]:
import cv2
import numpy as np
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Defining show image function to display image

In [ ]:
def show_image(img):    
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Take picture from web-cam

In [ ]:
def take_picture():   
    cap = cv2.VideoCapture(0)
    while 1:
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2,3)
        #print(faces)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
        cv2.imshow('img',img)

        if cv2.waitKey(1)==27:
            break

    cap.release()
    cv2.destroyAllWindows()
    return roi_color
#take_picture()

importing all required modules 

In [ ]:
from keras.layers import Dense,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation,Dropout

Defining Model

In [ ]:
def create_model():
    model=Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(210, 210, 3)))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

    
    model.add(Conv2D(128, (3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model
    

model=create_model()    

In [ ]:
model.summary()

Data Augmentation and reading data

In [ ]:
import tensorflow
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,
        fill_mode='nearest',
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
    rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,
        fill_mode='nearest',
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        './MASK_DETECTION/train/',
        target_size=(210, 210),
        class_mode='binary',
        batch_size=16)

validation_generator = test_datagen.flow_from_directory(
        './MASK_DETECTION/test/',
        target_size=(210, 210),
        class_mode='binary',
        batch_size=16)



Model fit

In [ ]:
model.fit_generator(
        train_generator,
        epochs=10,
        validation_data=validation_generator,)

In [ ]:
model.save("main.h5")

Live data prediction

In [ ]:
roi=take_picture()
#roi=cv2.imread('./Desktop/poen-cv/MASK_DETECTION/test/without_mask/9.jpg')
show_image(roi)
print(roi.shape)

resized=cv2.resize(roi,(210,210))
print(resized.shape)
show_image(resized)
s=np.reshape(resized,(1,210,210,3))

print(s.shape)
y=model.predict_classes(s)
print(y)

In [ ]:
from keras.models import load_model

In [ ]:
model=load_model('./main.h5')

In [ ]:
y=model.predict_classes(s)
print(y)

In [ ]:
train_generator.class_indices

In [ ]:
model.evaluate_generator(validation_generator)

Data Prediction On video

In [ ]:
def predict(img):
    
    clsno=model.predict_classes(img)
    return clsno[0][0]

In [ ]:
def detect_mask(img):
    gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_classifier=cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    faces=face_classifier.detectMultiScale(gray_img,1.1,3)
    faceimg=img
    if faces is ():
        print("No face Found")
        return img
    else:
        for ( x,y,w,h) in faces:
            r = max(w, h) / 2
            centerx = x + w / 2
            centery = y + h / 2
            xx = int(centerx - r)
            yy = int(centery - r)
            rr = int(r * 2)
            faceimg = img[yy:yy+rr, xx:xx+rr]
            faceimg=cv2.resize(faceimg,(210,210))
            faceimg=faceimg.reshape((1,210,210,3))
            faceimg=faceimg/255.0
            class_number=predict(faceimg)
            string=''
            
            if(class_number==0):
                string='Mask'
                new_img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
                new_img=cv2.putText(new_img,string,(x-10,y),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),2)
            else:
                string='No Mask'
                new_img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
                new_img=cv2.putText(new_img,string,(x-10,y),cv2.FONT_HERSHEY_PLAIN,3,(0,0,255),2)
            
                
    return new_img

In [ ]:
cap=cv2.VideoCapture(0)
while 1:
    ret,frame=cap.read()

    cv2.imshow("Video",detect(frame))
    if(cv2.waitKey(3)==27):
        break
cv2.destroyAllWindows()
cap.release()